In [ ]:
import pennylane as qml
from pennylane import numpy as np

In [ ]:
# Three qubits to use as graph nodes, one as coin.
node_count = 3
dev = qml.device("default.qubit", wires=node_count+1)

In [ ]:
def walk_step(node_count):
    # Flip the coin
    qml.Hadamard(wires=node_count)

    # Implement the Addition Operator

    qml.PauliX(wires=node_count)

    for i in range(node_count, 0, -1):
        controls = [v for v in range(node_count, i-1, -1)]
        qml.MultiControlledX(control_wires=controls, wires=i-1)
        if i > 1:
            qml.PauliX(wires=i-1)
    
    qml.PauliX(wires=node_count)

    # Implement the Substraction Operator
    for i in range(1, node_count+1):
        controls = [v for v in range(node_count, i-1, -1)]
        qml.MultiControlledX(control_wires=controls, wires=i-1)
        if i < node_count:
            qml.PauliX(wires=i)


@qml.qnode(dev)
def circuit(params):
    node_count = 3
    qml.PauliX(wires=0)
    qml.PauliX(wires=node_count)
    # This is the walk step
    for _ in range(params[0]):
        walk_step(node_count)
        
    # return qml.expval(qml.PauliZ(0))
    return [qml.sample(qml.PauliZ(i)) for i in range(node_count)]
    # return [qml.expval(qml.PauliZ(i)) for i in range(node_count)]


In [ ]:
def refined_results(x):
    result = circuit(x, shots=1000)
    result[result==-1] = 0
    return result.sum(axis=1)


In [94]:
def cost_result(result):
    return result[0]

In [ ]:
# Bias against the first qubit being 1.
def cost_fn(x):
    result = refined_results(x)
    return result[0]

In [ ]:
init_params = np.array([4, 0])
result = refined_results(init_params)
print(result)
print(cost_fn(init_params))

In [95]:
# set the number of steps
steps = 100
# set the initial parameter values
min_step, min_cost, min_result = -1, 1e9, None

for i in range(1, steps):
    # update the circuit parameters
    params = np.array([i, 0])
    
    result = refined_results(params)
    # calculate the cost
    cost = cost_result(result)

    if cost < min_cost:
        min_cost = cost
        min_step = i
        min_result = result

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost))

print("Optimized walk steps: {}".format(min_step))

Cost after step     5:  764.0000000
Cost after step    10:  264.0000000
Cost after step    15:  749.0000000
Cost after step    20:  257.0000000
Cost after step    25:  0.0000000
Cost after step    30:  738.0000000
Cost after step    35:  503.0000000
Cost after step    40:  744.0000000
Cost after step    45:  268.0000000
Cost after step    50:  503.0000000
Cost after step    55:  733.0000000
Cost after step    60:  496.0000000
Cost after step    65:  765.0000000
Cost after step    70:  238.0000000
Cost after step    75:  244.0000000
Cost after step    80:  739.0000000
Cost after step    85:  506.0000000
Cost after step    90:  744.0000000
Cost after step    95:  0.0000000
Cost after step   100:  748.0000000
Optimized walk steps: 0


In [ ]:
histo_steps = 1

params = np.array([min_step, 0])

for _ in range(histo_steps):
    result = circuit(params)
    print(result)